# Time

## On the philosophy of time

Starsim's implementation of time is complex, because, well, time is complex. Some questions we've wrestled with:

1. If you run a simulation from 2000 to 2020 with monthly timesteps, would you expect the same average result (say, number of infections) in 2004 (leap year) and 2005 (not a leap year)? What about February 2012 vs. March 2012?
2. Is a year 365 days, 365.25 days, or either 365 or 366 days depending on the year?
3. Usually years and dates are interchangeable, e.g. `float(2000) == ss.date('2000-01-01')`. So `start=2000, stop=2020` is equivalent to `start='2000-01-01', stop='2020-01-01'`. But what if someone wants to simply run for a number of years, e.g. `start=0, stop=20`, given that there is no year 0?
4. If the user has scheduled an event to happen at fractional year `2025.7`, do they mean the nearest day (`2025-09-12`) or do they mean that exact timestamp (`2025-09-11 16:48:00`)? Will users ever want to worry about hours and minutes? What about microseconds and nanoseconds?
5. Should units be strict, e.g. `ss.days(5) != 5`, or permissive, e.g. `ss.days(5) == 5`?
6. If a mortality rate is specified as "100 deaths per 1000 people per year", at the end of a year, should 100 people die ("100 deaths per 1000 people from the beginning to end of a year") or should 95 die ("100 deaths per 1000 **person**-years")? [See example below.]

Why are we telling you all this? Because although time seems intuitive, there are many corner cases where intuition breaks down (and, worse, it might not seem like it has broken down!). Thus, perhaps more than any other part of Starsim, be sure to *check your assumptions* about time. For example, in answer to the questions above, we've decided:

1. A month is defined as exactly 1/12th of a year, so there are the same number of infections in 2004 and 2005, and in February vs. March.
2. A year is equal to 365 days, although some stretching happens to make integer years and calendar dates line up where needed.
3. Because there is no way to represent year 0 with a date object (e.g. `ss.date()`, which is based on `pd.Timestamp`), for this (important) special case we need to switch to a different object, `ss.datedur()` (more on that below).
4. We assume the user means integer days, unless they have _explicitly_ requested otherwise (e.g. `dt=ss.days(0.1)`).
5. Units are loose when comparing to unitless quantities (`ss.days(5) == 5`), but are strict when comparing with other units (e.g. `ss.years(1) == ss.days(365)`).
6. In Starsim v2, we chose the former by default. In Starsim v3, we decided we were wrong to do that and now choose the latter.


In [5]:
"""
Sanity check of TimeProb vs InstProb
"""
import numpy as np

# Probability of an event happening over a year -- "TimeProb"
timeprob = 0.1

# "InstProb" -- "Instantaneous probability per year" (which makes no sense)
instprob = -np.log(1-timeprob)

year = 365
iprob_per_day = timeprob/year # Convert to instantaneous probability per day
did_not_happen = 1
for day in range(year):
    did_not_happen *= 1-iprob_per_day
happened = 1 - did_not_happen

print(f'{timeprob = }')
print(f'{instprob = }')
print(f'{happened*1000 = }')
# assert np.isclose(happened, timeprob, atol=1e-3)

timeprob = 0.1
instprob = np.float64(0.10536051565782628)
happened*1000 = 95.17497917664552
